In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import traceback
def getHTML(url,code='utf-8'):
    try:
        r=requests.get(url,timeout=30)
        r.raise_for_status()
        r.encoding=code
        return r.text
    except:
        return ""
def getStocklist(list,stock_url):
    html=getHTML(stock_url,'GB2312')
    soup=BeautifulSoup(html,'html.parser')
    a=soup.find_all('a')
    for i in a:
        try:
            href=i.attrs['href']
            list.append(re.findall(r"[s][hz]\d{6}",href)[0])
        except:
            continue
def getStockinfo(list,stock_url,fpath):
    count=0
    for stock in list:
        url=stock_url+stock+".html"
        html=getHTML(url)
        try:
            if html== "":
                continue
            infoDict={}
            soup=BeautifulSoup(html,'html.parser')
            stockinfo=soup.find('div',attrs={'class':'stock-bets'})
            name=stockinfo.find_all(attrs={'class':'bets-name'})[0]
            infoDict.update({'股票名称':name.text.split()[0]})
            keylist=stockinfo.find_all('dt')
            valuelist=stockinfo.find_all('dd')
            for i in range(len(keylist)):
                key=keylist[i].text
                value=valuelist[i].text
                infoDict[key]=value
       
            with open(fpath,'a',encoding='utf-8') as f:
                f.write(str(infoDict)+'\n')
                count=count+1
                print('\r当前速度:{:.2f}%'.format(count*100/len(list)),end='')
        except:
            count=count+1
            print('\r当前速度:{:.2f}%'.format(count*100/len(list)),end='')
            continue
            
    
    return ""
def main():
    shock_list_url='http://quote.eastmoney.com/stock_list.html'
    stock_info_url='https://gupiao.baidu.com/stock/'
    output_file='D://NewBaiduStockInfo1.txt'
    slist=[]
    getStocklist(slist,shock_list_url)
    getStockinfo(slist,stock_info_url,output_file)
main()